<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Interacting with File Shares in Storage

## Azure Actions Covered

* Uploading and downloading files from a file share
* Listing contents of a file share directory

In this lecture, we're going to take a look at how to interact with file shares in storage containers via the Python SDK.

To begin, we import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). 

In [1]:
# New imports for file shares
from azure.storage.fileshare import ShareClient, ShareServiceClient

from settings import AZURE_SUBSCRIPTION_ID, STORAGE_ACCESS_KEY

## Interacting with Files

First, let's set up our `ShareServiceClient` again for our storage account.

In [2]:
share_service_client = ShareServiceClient(
    account_url='https://benbstorage1234.file.core.windows.net',
    credential=STORAGE_ACCESS_KEY
)

Next, we can get the share client to interact with a specific share.

In [ ]:
share = share_service_client.get_share_client(share='bens-file-share')

To interact with files in our newly created file share, we need a file client (the `ShareFileClient()` class). We can retrieve that client using the `get_file_client()` method from our share object. All we need is a file name, and the file doesn't need to exist. We'll create a file client for a new file to upload.

In [11]:
file_client = share.get_file_client(file_path='python-test-file.txt')

Next, we'll use Python's context manager to open our source file and upload it via the file client.

In [12]:
with open('./python-test-file.txt', 'rb') as source_file:
    file_client.upload_file(source_file)

To add structure to our file share, we can create directories via the file share client as well. This will return a `ShareDirectoryClient` object.

In [13]:
directory = share.create_directory(directory_name='newdir')

We can see some of our directories properties as well.

In [14]:
directory.get_directory_properties()

{'name': None, 'last_modified': datetime.datetime(2023, 5, 15, 4, 8, 49, tzinfo=datetime.timezone.utc), 'etag': '"0x8DB54FA16306BCC"', 'server_encrypted': True, 'metadata': {}, 'change_time': datetime.datetime(2023, 5, 15, 4, 8, 49, 388641), 'creation_time': datetime.datetime(2023, 5, 15, 4, 8, 49, 388641), 'last_write_time': datetime.datetime(2023, 5, 15, 4, 8, 49, 388641), 'last_access_time': None, 'file_attributes': 'Directory', 'permission_key': '14617446708122078996*11673537244870367389', 'file_id': '11529285414812647424', 'parent_id': '0', 'is_directory': True}

It will also have a distinct URL under our file share account

In [15]:
directory.url

'https://benbstorage1234.file.core.windows.net/bens-file-share/newdir'

If you need to get the directory client for a directory that already exists, you can use `get_directory_client()`.

In [16]:
dir_client = share.get_directory_client(directory_path='newdir')

You can also get a file client from the directory client in order to create or interact with files under that directory. We'll try uploading our previous text file into a new directory.

In [17]:
file_client = dir_client.get_file_client(file_name='python-test-file.txt')

In [18]:
with open('./python-test-file.txt', 'rb') as source_file:
    file_client.upload_file(source_file)

We can now list the directories and files inside our file share as well, in case we need to understand the structure.

In [20]:
for file in share.list_directories_and_files(directory_name='newdir'):
    print(file)

{'name': 'python-test-file.txt', 'path': None, 'share': None, 'snapshot': None, 'content_length': None, 'metadata': None, 'file_type': None, 'last_modified': None, 'etag': None, 'size': 38, 'content_range': None, 'server_encrypted': None, 'copy': {'id': None, 'source': None, 'status': None, 'progress': None, 'completion_time': None, 'status_description': None, 'incremental_copy': None, 'destination_snapshot': None}, 'content_settings': {'content_type': None, 'content_encoding': None, 'content_language': None, 'content_md5': None, 'content_disposition': None, 'cache_control': None}, 'lease': {'status': None, 'state': None, 'duration': None}, 'change_time': None, 'creation_time': None, 'last_write_time': None, 'last_access_time': None, 'file_attributes': None, 'permission_key': None, 'file_id': '16140971433240035328', 'parent_id': None, 'is_directory': False}


Finally, we can get a file client by referencing a specific file in our share and then use that client to download the file.

In [22]:
file_client = share.get_file_client('python-test-file.txt')

We'll need to use a context manager again to download the file.

In [23]:
with open('downloaded-file.txt', 'wb') as my_file:
    data = file_client.download_file()
    data.readinto(my_file)

## Deleting Objects

To clean up our file share, we can delete files, directories, and the share. First, let's delete one of our created files with the `delete_file()` method. Because you've already specified the file name to create the client, you don't need that as a parameter.

In [24]:
file_client.delete_file()

You can delete an entire directory in the file share with the `delete_directory()` method. **Note:** you can't delete non-empty directories with this method. First, you'd have to delete all the files in the directory.

In [26]:
## share.delete_directory(directory_name='newdir')

Finally, we can clean up our storage account by deleting the file share we created.

In [27]:
share.delete_share()